In [ ]:
# Territory Factor Enhancement with Hexagons & Credibility

# --- 1. Libraries ---
import pandas as pd
import numpy as np
import h3
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import Point, Polygon
import geopandas as gpd

# --- 2. Load Data ---
df = pd.read_csv("territory_factors.csv")
print(df.head())

# --- 3. Credibility Adjustment ---
# Exposure credibility threshold
MIN_EXPOSURE = 50000

def apply_credibility(row, neighbors_df):
    if row['exposure'] >= MIN_EXPOSURE:
        return row['loss_cost']
    else:
        # Use neighbors' exposure & loss cost
        neighbors = neighbors_df.sample(3, replace=True, random_state=42)  # placeholder
        weights = neighbors['exposure'] / neighbors['exposure'].sum()
        return np.dot(neighbors['loss_cost'].fillna(0), weights)

df['cred_adj_loss_cost'] = df.apply(
    lambda r: apply_credibility(r, df), axis=1
)

# --- 4. Inverse Distance Weighting (for new ZIPs) ---
# Example: ZIP 46206 has no exposure/losses
target_zip = "46206"
target_row = df[df['zip_code'] == target_zip]

if target_row['exposure'].values[0] == 0:
    neighbors = df[df['exposure'] > 0].copy()
    neighbors['distance'] = np.random.uniform(1,10,len(neighbors))  # placeholder distances
    neighbors['weight'] = 1 / neighbors['distance']
    neighbors['weight'] = neighbors['weight'] / neighbors['weight'].sum()
    predicted_factor = np.dot(neighbors['cred_adj_loss_cost'], neighbors['weight'])
    df.loc[df['zip_code'] == target_zip, 'cred_adj_loss_cost'] = predicted_factor

# --- 5. Results ---
print(df[['zip_code','loss_cost','cred_adj_loss_cost']])

# --- 6. Visualization ---
sns.scatterplot(data=df, x='exposure', y='cred_adj_loss_cost', hue='zip_code')
plt.title("Credibility-Adjusted Loss Costs by Exposure")
plt.show()
